In [ ]:
!python --version

In [ ]:
!pip install beautifulsoup4;
!pip install html5lib;
!pip install lxml;
# TODO pip added

In [ ]:
import os.path
import shutil
import urllib.request as request
from contextlib import closing
from bs4 import BeautifulSoup

In [ ]:
# Source: https://www.data.gv.at/katalog/dataset/f3a1c18ea3a038fa13e7b3c89454ed280d16ef0c
file_url = "https://wahlen.tirol.gv.at/nationalratswahl_" # + "/dokumente/"
file_name_prefix = "wahl"
file_name_pstfix = ".csv"
file_local_location = "./"
file_save_pstfix = ".tsv"

In [ ]:
def download_dataset(year, file_name):
    current_file_name = file_name_prefix + file_name + file_name_pstfix
    file_store_url = file_local_location + current_file_name
    final_URL = file_url + year + "/dokumente/" + current_file_name
    
    if os.path.isfile(file_store_url):
        print(file_store_url)
        return file_store_url

    with closing(request.urlopen(final_URL)) as r:
        with open(file_store_url, 'wb') as f:
            shutil.copyfileobj(r, f)
            print(final_URL)
            return file_store_url

In [ ]:
def reformat_file(current_file_url):
    formated_data = {}
    with open(current_file_url, 'r', encoding="latin-1") as file:
        content = file.read()
        for region in content.split("\n")[2:]:
            current_values = region.split(";")
            if (len(current_values) > 13): # Has trailing empty line
                formated_data[current_values[2]] = {"Bezirk_Id":current_values[0]}            
                formated_data[current_values[2]]["Bezirk"] = current_values[1]
                formated_data[current_values[2]]["Gemeinde_Id"] = current_values[2]
                formated_data[current_values[2]]["Gemeinde"] = current_values[3]
                formated_data[current_values[2]]["Wahlberechtigte_maennlich"] = current_values[4]
                formated_data[current_values[2]]["Wahlberechtigte_weiblich"] = current_values[5]
                formated_data[current_values[2]]["Auslandsoesterreicher_Wahlberechtigte_maennlich"] = current_values[6]
                formated_data[current_values[2]]["Auslandsoesterreicher_Wahlberechtigte_weiblich"] = current_values[7]
                formated_data[current_values[2]]["Wahlberechtigte_gesamt"] = current_values[8]
                formated_data[current_values[2]]["Abgegebene_Stimmen"] = current_values[9]
                formated_data[current_values[2]]["Wahlbeteiligung"] = current_values[10]
                formated_data[current_values[2]]["Gueltige_Stimmen"] = current_values[11]
                formated_data[current_values[2]]["Ungueltige_Stimmen"] = current_values[12]
                formated_data[current_values[2]]["Parteien"] = {"ÖVP": 0}
                index = 0
                current_party = ""
                for v in current_values[13:]:
                    if index%4 == 1:
                        current_party = v
                        if current_party not in current_parties:
                            current_parties.append(current_party)
                    if index%4 == 2:
                        formated_data[current_values[2]]["Parteien"][current_party] = v
                    index += 1                
        return formated_data

In [ ]:
## main
number_of_files = {
    "1994":"6","1995":"7","1999":"8","2002":"10","2006": "18",
    "2008":"21","2013":"39","2017":"51","2019":"56"}
for i in number_of_files:
    
    content = "GKZ\tGebietsname\tWahlberechtigte\tAbgegebene\tUngültige\tGültige" # tab as split string
    current_file_url = download_dataset(i, number_of_files[i])
    current_parties = ["ÖVP", "SPÖ", "FPÖ"] 
    structured_content = reformat_file(current_file_url)
    for party in current_parties:
        content += "\t" + party
    content += "\n"
    for a in structured_content:
        content += "G" + structured_content[a]["Gemeinde_Id"] + "\t" + structured_content[a]["Gemeinde"] + "\t" +  structured_content[a]["Wahlberechtigte_gesamt"] + "\t" + structured_content[a]["Abgegebene_Stimmen"] + "\t" + structured_content[a]["Ungueltige_Stimmen"] + "\t" + structured_content[a]["Gueltige_Stimmen"]
        for party in current_parties:
            if (structured_content[a]["Parteien"][party] is None or party not in list(structured_content[a]["Parteien"].keys())):
                content += "\t0" # structured_content["70219"]["Parteien"]
                print(party)
            else:
                content += "\t" + structured_content[a]["Parteien"][party]
        content += "\n"
    with open(file_local_location + "NRW_Tirol_" + str(i) + file_save_pstfix, "w") as text_file:
        text_file.write(content)